# Retreiving and Preparing Stock's Income Statement Information
This section of the code will concentrate on extracting stock's income statement information from IEX Cloud API. At the end, this section of the code will provide a complete and formatted income statement dataset.  

## Importing neccessary modules

In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import xlsxwriter

## Setting up base URL

Fetching the fundamentals and finacial data about the stock from ther IEX Cloud endpoints.

In [3]:
from api_secret import IEX_CLOUD_API_TOKEN

symbol = 'FTNT'# input("Enther the ticker here: ")

# API endpoints to collect stock data
income_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/income?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'
financials_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/financials?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'
fundamentals_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/fundamentals?period=quarter&last=12&token={IEX_CLOUD_API_TOKEN}'

income_endpoint_data = requests.get(income_endpoint).json()
financials_endpoint_data = requests.get(financials_endpoint).json()
fundamentals_endpoint_data = requests.get(fundamentals_endpoint).json()

## Extracting the data
This part will concentrate on extracting the data that will form the final income statement for the stock.

### Forming the Dataset columns
The columns will be comprised of the Pandas DatetimeIndex. It will be created based on the quarters (reportDate) of the reported stock data.


First step is to find the quarters of the reported stock data.

In [4]:
reportDate_list = []
quarter_list = []

# Converting string date into the date object
for stock_data in income_endpoint_data['income']:
    converted_date = datetime.strptime(stock_data['reportDate'], '%Y-%m-%d').date()
    reportDate_list.append(converted_date)


# function to determine to which quarter the reported stock data
# is referred to
def quarter_finder(dates_list):
    global quarter_list

    for d in reportDate_list:
        q1 = date(d.year, 3, 31)
        q2 = date(d.year, 6, 30)
        q3 = date(d.year, 9, 30)
        q4 = date(d.year, 12, 31)

        if d < q1:
            q = date(d.year - relativedelta(year=1).year, 12, 31)
        if d > q1 and d < q2:
            q = date(d.year, 3, 30)
        if d > q2 and d < q3:
            q = date(d.year, 6, 30)
        if d > q3 and d < q4:
            q = date(d.year, 9, 30)
        
        quarter_list.append(q)

quarter_finder(reportDate_list)

The above created Datetime-like array will serve as an input to Pandas' DatetimeIndex objet (columns of the future dataset)

In [5]:
columns = pd.DatetimeIndex(data=quarter_list)

The next step is the creation of the actual DataFrame.

In [6]:
index = [
    'Total Revenue',
    'Total Revenue (Growth)',
    'Cost of Revenue',
    'Cost of Revenue (Growth)',
    'Gross Profit',
    'Gross Profit (Growth)',
    'Operating Expense',
    'SG&A',
    'SG&A (Growth)',
    'Research and Development',
    'Operating Income',
    'Pretax Income',
    'Pretax Income (Growth)',
    'Tax Provision',
    'Net Income',
    'Net Income (Growth)',
    'Basic EPS',
    'Basic EPS (Growth)',
    'Diluted EPS',
    'Total Expenses',
    'Total Expenses (Growth)',
    'Interest Income',
    'Inerest Income (Growth)',
    'Interest Expense',
    'Interest Expense (Growth)',
    'EBIT',
    'EBITDA',
    'EBITDA (Growth)'
]

income_statement = pd.DataFrame(columns=columns, index=index, data=None)

### Filling the dataframe with data returned by API endpoints.

The first API endpoint to get the stock data from is the 'financials' endpoint.

In [7]:
financials_dict = {
    'Total Revenue': [],
    'Operating Income': [],
    'Pretax Income': [],
    'Tax Provision': [],
    'Net Income': [],
    'Total Expenses': [],
    'Interest Income': [],
    'EBIT': [],
    'EBITDA': []
}


# Filling the financials_dict dictionary with data collected
# from the API endpoint
for quarter_data in financials_endpoint_data['financials']:
    financials_dict['Total Revenue'].append(quarter_data['totalRevenue'])
    financials_dict['Operating Income'].append(quarter_data['operatingIncome'])
    financials_dict['Pretax Income'].append(quarter_data['pretaxIncome'])
    financials_dict['Tax Provision'].append(quarter_data['incomeTax'])
    financials_dict['Net Income'].append(quarter_data['netIncome'])
    financials_dict['Total Expenses'].append(quarter_data['operatingExpense'])
    financials_dict['Interest Income'].append(quarter_data['interestIncome'])
    financials_dict['EBIT'].append(quarter_data['ebit'])
    financials_dict['EBITDA'].append(quarter_data['EBITDA'])


# Putting the data into the DataFrame
for stock_attribute in financials_dict:
    for row in income_statement.index:
        if row == stock_attribute:
            income_statement.loc[row] = financials_dict[stock_attribute]


The second is the income API endpoint.

In [8]:
income_dict = {
    'Cost of Revenue': [],
    'Gross Profit': [],
    'Operating Expense': [],
    'SG&A': [],
    'Research and Development': []
}

for quarter_data in income_endpoint_data['income']:
    income_dict['Cost of Revenue'].append(quarter_data['costOfRevenue'])
    income_dict['Gross Profit'].append(quarter_data['grossProfit'])
    income_dict['Operating Expense'].append(quarter_data['operatingExpense'])
    income_dict['SG&A'].append(quarter_data['sellingGeneralAndAdmin'])
    income_dict['Research and Development'].append(quarter_data['researchAndDevelopment'])

for stock_attribute in income_dict:
    for row in income_statement.index:
        if row == stock_attribute:
            income_statement.loc[row] = income_dict[stock_attribute]

The final endpoint to use the data from in the dataset is the fundamentals endpoint.

In [9]:
fundamentals_dict = {
    'Basic EPS': [],
    'Diluted EPS': [],
    'Interest Expense': []
}

for quarter_data in fundamentals_endpoint_data['fundamentals']:
    fundamentals_dict['Basic EPS'].append(quarter_data['incomeNetPerWabsoSplitAdjusted'])
    fundamentals_dict['Diluted EPS'].append(quarter_data['incomeNetPerWadsoSplitAdjusted'])
    fundamentals_dict['Interest Expense'].append(quarter_data['expensesInterest'])

for stock_attribute in fundamentals_dict:
    for row in income_statement.index:
        if row == stock_attribute:
            income_statement.loc[row] = fundamentals_dict[stock_attribute]

## Calculating the Growth

Below section deals with the 'Growth' columns to give an investor an advantage of company's success metric.

In [10]:
# Calculating the growth rate based on the stock metrics' 
for df_index, row_name in enumerate(income_statement.index):
    if 'Growth' in row_name:
        for row_index, value in enumerate(income_statement.iloc[df_index - 1]):
            try:
                growth_rate = (value / income_statement.iloc[df_index - 1, row_index + 1]) * 100
                income_statement.iloc[df_index, row_index] = (growth_rate - 100) / 100
            except IndexError:
                break

income_statement

,2022-09-30,2022-06-30,2022-03-30,2021-12-31,2021-09-30,2021-06-30,2021-03-30,2020-12-31,2020-09-30,2020-06-30,2020-03-30,2019-12-31
Total Revenue,1153141284,1041626429,962254102,1000580547,897447949,822464758,719167406,768856017,665030576,633847164,590884169,624905291
Total Revenue (Growth),0.107058,0.082486,-0.038304,0.114918,0.091169,0.143635,-0.064627,0.156121,0.049197,0.07271,-0.054442,NaN
Cost of Revenue,294808303,262673503,262943619,238740272,212173228,192009196,158298066,174111909,140239495,138514225,133843328,145814440
Cost of Revenue (Growth),0.122337,-0.001027,0.101379,0.125214,0.105016,0.21296,-0.090826,0.241533,0.012456,0.034898,-0.082098,NaN
Gross Profit,898988594,810705617,719278520,757531326,683051281,633472738,577840225,607467310,522143953,498634254,462775143,476123363
Gross Profit (Growth),0.108896,0.127109,-0.050497,0.10904,0.078265,0.096277,-0.048771,0.16341,0.047148,0.077487,-0.028035,NaN
Operating Expense,917510868,876921606,840935503,781396323,716315605,657009216,593693745,601937313,536819375,500731496,513557934,519439929
SG&A,490773030,471205727,430541721,409243937,399063819,370160024,348006970,328761945,306086684,292892613,290340114,296649005
SG&A (Growth),0.041526,0.094448,0.052042,0.02551,0.078085,0.063657,0.058538,0.074081,0.045047,0.008791,-0.021267,NaN
Research and Development,138782252,126617723,130877593,115546244,110895471,111600958,99591199,90093622,93267750,82531407,82719141,71774049


Dealing with NaN values.

In [11]:
income_statement.fillna(value='', inplace=True)